email agent
- authenticates user
    - only then are they allowed into the "inbox"
    - dynamic tools and prompt on the condition of there being an email and password in state that match hardcoded
- checks "inbox"
    - email in tool
- sends emails
    - human in the loop

In [1]:
# Import required modules
from dotenv import load_dotenv

import operator
from typing import Annotated, TypedDict, Literal, List, Union
from dataclasses import dataclass, asdict

from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage, BaseMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import InjectedState

In [2]:
# load_dotenv
load_dotenv()

True

In [3]:
# Setup Model
model = ChatOllama(model="llama3.1:8b", temperature=0)

In [4]:
# Define Context & State
@dataclass
class EmailContext:
    email_address: str = "julie@example.com"
    password: str = "password123"

class AuthenticatedState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    authenticated: bool
    context: dict  # Dict for safe Pydantic serialization

In [5]:
# Define Tools
@tool
def authenticate(email: str, password: str, state: Annotated[dict, InjectedState]) -> str:
    """Authenticate the user with email and password. REQUIRED before checking inbox."""
    context = state.get("context", {})
    if email == context.get("email_address") and password == context.get("password"):
        return "SUCCESS: You are now authenticated."
    return "FAILURE: Incorrect credentials. Please try again."

@tool
def check_inbox() -> str:
    """Check the inbox for recent emails."""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send a response email to a recipient."""
    return f"Email successfully sent to {to}."

@tool
def logout() -> str:
    """Log the user out and clear the current session."""
    return "SUCCESS: You have been logged out."

In [6]:
# Graph Nodes and Logic
def call_model(state: AuthenticatedState):
    """Dynamically binds tools and sets system prompts based on auth status."""
    is_auth = state.get("authenticated", False)
    
    if is_auth:
        sys_msg = SystemMessage(content=(
            "You are AUTHENTICATED. You can check_inbox or send_email.\n"
            "When asked to reply to an email:\n"
            "1. Generate THREE distinct draft options (Professional, Friendly, and Concise).\n"
            "2. Present them to the user clearly numbered (1, 2, 3).\n"
            "3. DO NOT call the 'send_email' tool yet. Wait for the user to tell you which number they prefer.\n"
            "4. Only after the user picks a number should you call 'send_email' with that draft's content."
            "The system will pause for user approval before sending."
        ))
        tools = [check_inbox, send_email, logout]
    else:
        sys_msg = SystemMessage(content=(
            "You are UNAUTHENTICATED. You MUST call 'authenticate' with the "
            "provided email and password before doing anything else."
        ))
        tools = [authenticate]
    
    llm_with_tools = model.bind_tools(tools)
    response = llm_with_tools.invoke([sys_msg] + state["messages"])
    return {"messages": [response]}


def auth_checker(state: AuthenticatedState):
    """Updates authentication status based on tool results."""
    last_message = state["messages"][-1]
    
    if isinstance(last_message, ToolMessage):
        if "SUCCESS: You are now authenticated" in last_message.content:
            return {"authenticated": True}
        if "SUCCESS: You have been logged out" in last_message.content:
            return {"authenticated": False}
            
    return {}

In [7]:
# Build the Graph
workflow = StateGraph(AuthenticatedState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", ToolNode([authenticate, check_inbox, send_email, logout]))
workflow.add_node("auth_checker", auth_checker)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", tools_condition)
workflow.add_edge("tools", "auth_checker")
workflow.add_edge("auth_checker", "agent")

# Memory for thread persistence and HITL interrupts
memory = InMemorySaver()
app = workflow.compile(checkpointer=memory, interrupt_before=["tools"])

In [8]:
# Execution Helper
config = {"configurable": {"thread_id": "global_thread_1"}}
context_data = asdict(EmailContext())

def run_agent(user_input: str):
    print(f"\n--- Processing: {user_input} ---")
    
    # 1. Start the stream
    events = app.stream(
        {"messages": [HumanMessage(content=user_input)], "context": context_data},
        config,
        stream_mode="values"
    )

    for event in events:
        if "messages" in event:
            event["messages"][-1].pretty_print()

    # 2. Handle Interruption Logic
    snapshot = app.get_state(config)
    
    # While there is a 'next' node to run, and that node is 'tools'
    while snapshot.next and snapshot.next[0] == "tools":
        last_msg = snapshot.values["messages"][-1]
        
        # Check if the tool calls contain a sensitive action
        tool_calls = getattr(last_msg, 'tool_calls', [])
        is_sensitive = any(tc['name'] == 'send_email' for tc in tool_calls)
        
        if is_sensitive:
            print("\n" + "!"*30)
            print("HUMAN-IN-THE-LOOP REQUIRED: SEND EMAIL")
            print("!"*30)
            
            choice = input("Do you approve sending this email? (yes/no): ")
            if choice.lower() == "yes":
                print("\nResuming: Sending email...")
                # Pass None to resume the existing thread
                for event in app.stream(None, config, stream_mode="values"):
                    if "messages" in event:
                        event["messages"][-1].pretty_print()
                snapshot = app.get_state(config) # Update snapshot after resume
            else:
                print("\nAction cancelled. Feedback provided to agent.")
                # Optional: Send a rejection message back to the graph
                # app.update_state(config, {"messages": [HumanMessage(content="I reject this email draft. Please change the time to 11am.")]})
                break 
        else:
            # AUTO-APPROVE safe tools (authenticate, check_inbox)
            print(f"\n[Auto-Approving safe tool: {tool_calls[0]['name']}]")
            for event in app.stream(None, config, stream_mode="values"):
                if "messages" in event:
                    event["messages"][-1].pretty_print()
            snapshot = app.get_state(config) # Update snapshot after auto-resume

In [9]:
# Login and Check
run_agent("login as julie@example.com / password123 and check my inbox")


--- Processing: login as julie@example.com / password123 and check my inbox ---
================================ Human Message =================================

login as julie@example.com / password123 and check my inbox
================================== Ai Message ==================================
Tool Calls:
  authenticate (1466ff9a-953a-48c0-8640-b588c4efa827)
 Call ID: 1466ff9a-953a-48c0-8640-b588c4efa827
  Args:
    email: julie@example.com
    password: password123

[Auto-Approving safe tool: authenticate]
================================== Ai Message ==================================
Tool Calls:
  authenticate (1466ff9a-953a-48c0-8640-b588c4efa827)
 Call ID: 1466ff9a-953a-48c0-8640-b588c4efa827
  Args:
    email: julie@example.com
    password: password123
================================= Tool Message =================================
Name: authenticate

SUCCESS: You are now authenticated.
================================= Tool Message =================================
Nam

In [10]:
run_agent("check_inbox")


--- Processing: check_inbox ---
================================ Human Message =================================

check_inbox
================================== Ai Message ==================================
Tool Calls:
  check_inbox (ccba7e7b-8450-4d9a-8f27-7f77d51f6108)
 Call ID: ccba7e7b-8450-4d9a-8f27-7f77d51f6108
  Args:

[Auto-Approving safe tool: check_inbox]
================================== Ai Message ==================================
Tool Calls:
  check_inbox (ccba7e7b-8450-4d9a-8f27-7f77d51f6108)
 Call ID: ccba7e7b-8450-4d9a-8f27-7f77d51f6108
  Args:
================================= Tool Message =================================
Name: check_inbox


    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    
================================== Ai Message ==================================

You have 1 new email. Would you like to reply to it? 

Here are three draft options:

1. Professional:
"Hi J

In [11]:
run_agent("2")


--- Processing: 2 ---
================================ Human Message =================================

2
================================== Ai Message ==================================
Tool Calls:
  send_email (180e5522-414a-4aa2-a050-5cc14e5f5e09)
 Call ID: 180e5522-414a-4aa2-a050-5cc14e5f5e09
  Args:
    body: Hey Jane! I’m so glad you’re coming to town! Let’s grab that coffee and catch up on everything we’ve missed. Looking forward to seeing you soon! Hugs, Julie
    subject: Re: Hi Julie
    to: jane@example.com

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
HUMAN-IN-THE-LOOP REQUIRED: SEND EMAIL
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


Do you approve sending this email? (yes/no):  yes



Resuming: Sending email...
================================== Ai Message ==================================
Tool Calls:
  send_email (180e5522-414a-4aa2-a050-5cc14e5f5e09)
 Call ID: 180e5522-414a-4aa2-a050-5cc14e5f5e09
  Args:
    body: Hey Jane! I’m so glad you’re coming to town! Let’s grab that coffee and catch up on everything we’ve missed. Looking forward to seeing you soon! Hugs, Julie
    subject: Re: Hi Julie
    to: jane@example.com
================================= Tool Message =================================
Name: send_email

Email successfully sent to jane@example.com.
================================== Ai Message ==================================

Your inbox is still empty. Would you like to check it again or perform another action?


In [ ]:
run_agent("logout")


--- Processing: logout ---
================================ Human Message =================================

logout
================================== Ai Message ==================================
Tool Calls:
  logout (f53fe082-5885-4edf-8133-bb50297524f3)
 Call ID: f53fe082-5885-4edf-8133-bb50297524f3
  Args:

[Auto-Approving safe tool: logout]
================================== Ai Message ==================================
Tool Calls:
  logout (f53fe082-5885-4edf-8133-bb50297524f3)
 Call ID: f53fe082-5885-4edf-8133-bb50297524f3
  Args:
================================= Tool Message =================================
Name: logout

SUCCESS: You have been logged out.
================================= Tool Message =================================
Name: logout

SUCCESS: You have been logged out.
